<a href="https://colab.research.google.com/github/hazmicokro/Mini-Search-Engine-Study-Case-News-Question-About-COVID-19/blob/main/Mini%20Search%20Engine%20(Study%20Case%20%3A%20News%20Question%20About%20COVID-19).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nama : Mochammad Hazmi Cokro Mandiri**

**NIM : 201710370311240**

**Temu Kembali Informasi 7A**

Dataset : [NEWS](https://www.kaggle.com/xhlulu/covidqa?select=news.csv) dari .csv File QnA Kaggle 

**NEWS QUESTION COVID-19 -  SEARCH ENGINE**

Penilaian :
1. Tahapan yang harus ada : crawling, preprocessing, indexing, retrieval, perankingan.
2. Link URL minimal 10 links.
3. Preprocessing : tokenizing, stemming, stopword removal, case folding.
4. Indexing : term, dan ada pembobotan (bisa pakai tfidf atau metode pembobotan lain).
5. Retrieval : menggunakan cosine similarity atau improvisasi metode lain.
6. perankingan berdasarkan nilai similarity
7. Output seperti search engine (ada daftar link yang bisa di klik)

Poin plus :
Jika ada improvisasi => bisa dari metode pembobotan atau retrieval, studi kasus/dokumen yang di retrieve, metode perankingan, atau bisa juga dari metode preprocessing.

#[1] Download Dataset (Crawling)

## Mount dan Lokasi Kerja

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Tubes Search Engine - 240"

/content/drive/MyDrive/Tubes Search Engine - 240


##Download Dataset

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Tubes Search Engine - 240"

In [ ]:
from google.colab import files
files.upload()
print("Done Upload Kaggle.json")

Done Upload Kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle

!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [ ]:
!kaggle datasets download -d xhlulu/covidqa

100% 1.77G/1.78G [00:18<00:00, 114MB/s]
100% 1.78G/1.78G [00:18<00:00, 105MB/s]


In [ ]:
!unzip covidqa.zip

Archive:  covidqa.zip
  inflating: news.csv                


##Load Dataset

In [ ]:
import pandas as pd 

df = pd.read_csv("/content/drive/MyDrive/Tubes Search Engine - 240/news.csv")
with pd.option_context('display.max_rows',10):
    display(df)

,source,url,question,answer,wrong_answer
0,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,What are the symptoms?,"Symptoms include fever, coughing, sore throat,...","As cases of coronavirus surge, health authorit..."
1,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,When should I get tested?,Your doctor will tell you if you need to get t...,If you have been in contact with a person with...
2,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,What's the difference between physical distanc...,"As cases of coronavirus surge, health authorit...","Remember, the vast majority of Australians are..."
3,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,How do I practice physical distancing?,If you have been in contact with a person with...,Other countries have closed schools and univer...
4,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,What's closed?,Physical distancing is the reason the Federal ...,"Symptoms include fever, coughing, sore throat,..."
...,...,...,...,...,...
476,USA Today,https://www.usatoday.com/story/news/health/202...,Is the presence of a fever always the first sy...,Symptoms of COVID-19 resemble the flu and incl...,"No, for several reasons, says Peter Hotez, dea..."
477,USA Today,https://www.usatoday.com/story/news/health/202...,How can I know if I have the coronavirus and s...,"If you have symptoms and want to get tested, t...",No. While shortness of breath is among the mos...
478,USA Today,https://www.usatoday.com/story/news/health/202...,Does testing positive mean that I have the vir...,"Yes, testing positive means that you have the ...",You should take precaution with any containers...
479,USA Today,https://www.usatoday.com/story/news/health/202...,"There has been a Hong Kong flu, a Swine flu, H...",The current outbreak is the world's first pand...,"Vaccines against pneumonia, such as pneumococc..."


#[2] Preprocessing Dataset

**Drop Wrong Answer (Fitur yang kurang berkorelasi dengan fitur lain)**

In [ ]:
df.drop('wrong_answer', axis=1, inplace=True)
df.head()

,source,url,question,answer
0,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,What are the symptoms?,"Symptoms include fever, coughing, sore throat,..."
1,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,When should I get tested?,Your doctor will tell you if you need to get t...
2,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,What's the difference between physical distanc...,"As cases of coronavirus surge, health authorit..."
3,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,How do I practice physical distancing?,If you have been in contact with a person with...
4,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,What's closed?,Physical distancing is the reason the Federal ...


**Drop NaN / na / null value**

In [ ]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 479 entries, 0 to 480
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   source    479 non-null    object
 1   url       479 non-null    object
 2   question  479 non-null    object
 3   answer    479 non-null    object
dtypes: object(4)
memory usage: 18.7+ KB


###Case Folding
**Mengubah huruf Kapital menjadi lower_case (kecil)**

In [ ]:
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df['question'] = df['question'].apply(lambda x: lower_case(x))
df['answer'] = df['answer'].apply(lambda x: lower_case(x))
df.head(3)

,source,url,question,answer
0,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,what are the symptoms?,"symptoms include fever, coughing, sore throat,..."
1,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,when should i get tested?,your doctor will tell you if you need to get t...
2,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,what's the difference between physical distanc...,"as cases of coronavirus surge, health authorit..."


###Tokenizing 
**Menghapus Simbol dan Angka , Melakukan Word Tokenize**

In [ ]:
import re

df['question'] = df['question'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df['answer'] = df['answer'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df.head(3)

,source,url,question,answer
0,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,what are the symptoms,symptoms include fever coughing sore throat fa...
1,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,when should i get tested,your doctor will tell you if you need to get t...
2,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,whats the difference between physical distanci...,as cases of coronavirus surge health authoriti...


In [ ]:
import nltk
nltk.download('punkt')

df["question"] = df.apply(lambda row: nltk.word_tokenize(row["question"]), axis=1)
df["answer"] = df.apply(lambda row: nltk.word_tokenize(row["answer"]), axis=1)

df.head(3)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,source,url,question,answer
0,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,"[what, are, the, symptoms]","[symptoms, include, fever, coughing, sore, thr..."
1,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,"[when, should, i, get, tested]","[your, doctor, will, tell, you, if, you, need,..."
2,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,"[whats, the, difference, between, physical, di...","[as, cases, of, coronavirus, surge, health, au..."


###Stemming 
**Mengubah Kata tidak baku menjadi Kata Baku**

In [ ]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
df['question'] = df['question'].apply(lambda x: [stemmer.stem(y) for y in x])
df['answer'] = df['answer'].apply(lambda x: [stemmer.stem(y) for y in x])
df.head(3)

,source,url,question,answer
0,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,"[what, are, the, symptom]","[symptom, includ, fever, cough, sore, throat, ..."
1,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,"[when, should, i, get, test]","[your, doctor, will, tell, you, if, you, need,..."
2,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,"[what, the, differ, between, physic, distanc, ...","[as, case, of, coronavirus, surg, health, auth..."


###Stop Words
**Menghapus kata imbuhan**

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop = set(stopwords.words('english'))
exclude = set(('what', 'where', 'when', 'who', 'why' ,'how' , 'i', 'we','us'))
stop_words = stop.difference(exclude)
print(stop_words)

def remove_stops1(row):
    my_list1 = row['question']
    meaningful_words1 = [w for w in my_list1 if not w in stop_words]
    return (meaningful_words1)

def remove_stops2(row):
    my_list2 = row['answer']
    meaningful_words2 = [w for w in my_list2 if not w in stop_words]
    return (meaningful_words2)

df['question'] = df.apply(remove_stops1, axis=1)
df['answer'] = df.apply(remove_stops2, axis=1)

df.head(3)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{"weren't", 'below', 'both', 'or', 't', 'ours', 'no', 'the', 'after', 'because', 'some', 'too', "hasn't", "won't", 'such', 'of', 'those', 'again', 'on', "you've", 'needn', 'itself', 'won', 'her', 'been', 'if', 'and', 'that', 'shan', 'hers', 'ma', 'into', 'he', 'as', 'wasn', 'himself', "doesn't", 'am', 'up', 'once', 'yourself', 'themselves', 'here', "mightn't", 'to', 'now', "wasn't", 'but', 'all', 'o', 'm', 'while', 'hadn', 'not', "mustn't", 'so', 'own', 'will', 'she', 'whom', 'you', 'll', "she's", 'hasn', 'be', 'his', 've', 'does', 'any', 'few', 'off', 'mustn', "needn't", "you'll", 'other', 'being', 'have', "shouldn't", 'further', 'between', 'nor', 'there', 'it', 'above', 'couldn', 'at', 'were', 'herself', "wouldn't", 'has', 'only', 'had', 'their', 'don', "aren't", 'doesn', "haven't", 'which', 'very', 'having', 'should', 'd', 'do', "shan't", 'did', 'before', 'mightn', 

,source,url,question,answer
0,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,"[what, symptom]","[symptom, includ, fever, cough, sore, throat, ..."
1,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,"[when, i, get, test]","[doctor, tell, need, get, test]"
2,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,"[what, differ, physic, distanc, selfisol]","[case, coronavirus, surg, health, author, advi..."


###Re Join Words

**Menggabungkan ulang kata yang telah di preprocessing ke kalimat awal**

In [ ]:
def rejoin_words1(row):
    my_list1 = row['question']
    joined_words1 = ( " ".join(my_list1))
    return joined_words1

def rejoin_words2(row):
    my_list2 = row['answer']
    joined_words2 = ( " ".join(my_list2))
    return joined_words2

df['question'] = df.apply(rejoin_words1, axis=1)
df['answer'] = df.apply(rejoin_words2, axis=1)

df.head(3)

,source,url,question,answer
0,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,what symptom,symptom includ fever cough sore throat fatigu ...
1,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,when i get test,doctor tell need get test
2,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,what differ physic distanc selfisol,case coronavirus surg health author advis aust...


#[3] Indexing / Pembobotan TF-IDF
**TF-IDF Vectorize Sklearn**

In [ ]:
# Library TF-IDF Vectorize Sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance

### TF - IDF
**Term Frequency - Inverse Document Frequency**

In [ ]:
# Membuat Fungsi TF IDF menggunakan Library Sklearn
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['question'])
print(X.shape)

(479, 739)


#[4] Retrieval / Cosine Similarity
**Cosine Similarity antar query dan data**

In [ ]:
print('Masukkan Kata Kunci Pencarian : ')
query = input()

# Query Input di Vectorize dan dihitung nilai cosine similarity
query_vec = vectorizer.transform([query]) 
results = cosine_similarity(X,query_vec).reshape(-1)

df['cos_sim'] = cosine_similarity(X, query_vec).reshape(-1)
with pd.option_context('display.max_rows',10):
    display(df)

Masukkan Kata Kunci Pencarian : 
how to know coronavirus


,source,url,question,answer,cos_sim
0,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,what symptom,symptom includ fever cough sore throat fatigu ...,0.000000
1,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,when i get test,doctor tell need get test,0.000000
2,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,what differ physic distanc selfisol,case coronavirus surg health author advis aust...,0.000000
3,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,how i practic physic distanc,contact person covid19 infect ill need isol 14...,0.086971
4,ABC Australia,https://www.abc.net.au/triplej/programs/hack/c...,what close,physic distanc reason feder govern close pub i...,0.000000
...,...,...,...,...,...
476,USA Today,https://www.usatoday.com/story/news/health/202...,presenc fever alway first symptom coronavirus ...,symptom covid19 resembl flu includ fever cough...,0.060106
477,USA Today,https://www.usatoday.com/story/news/health/202...,how i know i coronavirus go test,symptom want get test cdc recommend call state...,0.757903
478,USA Today,https://www.usatoday.com/story/news/health/202...,doe test posit mean i virus i develop symptom,yes test posit mean virus doe mean develop sym...,0.000000
479,USA Today,https://www.usatoday.com/story/news/health/202...,hong kong flu swine flu h1n1 sar mer ebola zik...,current outbreak world first pandem caus coron...,0.086961


#[5] Perankingan / TOP 10

In [ ]:
print('Kata Kunci Pencarian :  ' + query, "\n")
print("Top 10 - Pencarian yang serupa dengan kata kunci")
print("--------------------------------------------------------------------")

for i in results.argsort()[-10:][::-1]:
    print("Hasil Pencarian \t: ",df.iloc[i,2],
              "\nCosine Similarity \t: ",df.iloc[i,4],
              "\nPortal Berita \t\t: ",df.iloc[i,0],
              "\nLink Berita \t\t: ",df.iloc[i,1])
    print(" ")

Kata Kunci Pencarian :  how to know coronavirus 

Top 10 - Pencarian yang serupa dengan kata kunci
--------------------------------------------------------------------
Hasil Pencarian 	:  how i know i coronavirus 
Cosine Similarity 	:  1.0 
Portal Berita 		:  ABC News 
Link Berita 		:  https://www.abc.net.au/news/2020-03-13/coronavirus-faq-covid-19-frequently-asked-questions/12042318
 
Hasil Pencarian 	:  how i know i coronavirus go test 
Cosine Similarity 	:  0.7579033832948884 
Portal Berita 		:  USA Today 
Link Berita 		:  https://www.usatoday.com/story/news/health/2020/03/12/coronavirus-reader-questions-answered-symptoms-testing-elderly/5012457002/
 
Hasil Pencarian 	:  how i know i coronavirus go test 
Cosine Similarity 	:  0.7579033832948884 
Portal Berita 		:  USA Today 
Link Berita 		:  https://www.usatoday.com/story/news/health/2020/03/12/coronavirus-reader-questions-answered-symptoms-testing-elderly/5012457002/
 
Hasil Pencarian 	:  how know cold flu covid19 
Cosine Similarit